**A Naive Bayes Way for Spam Classification using Tabu List**
=============================================================
Implemented by Clyde Wang, Feb, 17, 2017

This spam classifier is implemented by Naive Bayes Model, a simple but very efficient solution in spam classification problem. In brief, Naive Bayes treats every features independent from each other, making inference very efficient. You can refer to [Naive Bayes][1] for more details. This article briefly introduces the process of the selection of tabu list and building the learning algorithm. I hope you will enjoy it.

This code runs quite well with the accuracy of 98.31% on training sample and 97.81% on test sample.
I hope someone could improve it and enhance its performance. And Here we go!

  [1]: https://en.wikipedia.org/wiki/Naive_Bayes_classifier

## 1. Environment Setting ##
Before we start we need set up the environment, please make sure those packages are installed in your computer when you copy this code to the local file.

In [ ]:
#coding:utf-8
import pandas as pd
import numpy as np
import re
from sklearn.naive_bayes import BernoulliNB

2. Read Data File
-----------------
The first task is to read data from .csv file, and here we could take advantage of the `pandas.DataFrame` to store and process the raw data. (see [here][1]) Notice that we have to split our raw data into two parts in order to test the generalisation ability of our model. Here is the code:


  [1]: http://pandas.pydata.org/pandas-docs/stable/10min.html

In [ ]:
def readData():
	SMS_df = pd.read_csv('../input/spam.csv',usecols=[0,1],encoding='latin-1')
	SMS_df.columns=['label','content']
	n = int(SMS_df.shape[0])
    # split into training data and test data
	return SMS_df.iloc[:int(n/2)], SMS_df.iloc[int(n/2):]

3. Generate a Tabu List
--------------------
A **tabu list** is a list of those significant indicators of spam SMS. Here we select TF-IDF as the principle of list generation.

Term Frequency(TF) is the frequency of a word in a certain kind of document. If there is a article of 50 words with 2 'data' in it, then the TF of the 'data' is given by `2/50=0.01`.

However, there are some words of high frequency in English, like 'a', 'is', 'are', etc. We have to remove those words from our list. And here comes the IDF.

Inverse Document Frequency(IDF) is the indicator to reflect how important a word is related to some certain topic. It is given by `log(#total articles/#articles containing w)`, for example, if we have 5 articles, only one has word 'gene' of term frequency of 0.002, but all the five articles contains the word 'technology' of term frequency of 0.5, the IDF of 'gene' is `log(5/1)>0` but  the IDF of 'technology' is `log(5/5)=0`.

TF-IDF is the product of TF and IDF: in the example above, `TFIDF('gene')=0.002*log(5/1)>0` while `TFIDF('technology')=0.5*log(5/5)=0`, so in this case, 'gene' is a better indicator than 'technology'.

In my code, I compute the TF-IDF of each word for 'spam' and 'ham', and compute the difference between them, thus I can select the words which are the most representative for the 'spam' class.

Here is the code:

In [ ]:
def generate_tabu_list(path, tabu_size=200,ignore=3):
	train_df,_ = readData()
	spam_TF_dict = dict()
	ham_TF_dict = dict()
	IDF_dict = dict()

	# ignore all other than letters.
	for i in range(train_df.shape[0]):
		finds = re.findall('[A-Za-z]+', train_df.iloc[i].content)
		if train_df.iloc[i].label == 'spam':
			for find in finds:
				if len(find)<ignore: continue
				find = find.lower()
				try:
					spam_TF_dict[find] = spam_TF_dict[find] + 1
				except:	
					spam_TF_dict[find] = spam_TF_dict.get(find,1)
					ham_TF_dict[find] = ham_TF_dict.get(find,0)
		else:
			for find in finds:
				if len(find)<ignore: continue
				find = find.lower()
				try:
					ham_TF_dict[find] = ham_TF_dict[find] + 1
				except:	
					spam_TF_dict[find] = spam_TF_dict.get(find,0)
					ham_TF_dict[find] = ham_TF_dict.get(find,1)
		
		word_set = set()
		for find in finds:
			if len(find)<ignore: continue
			find = find.lower()
			if not(find in word_set):
				try:
					IDF_dict[find] = IDF_dict[find] + 1
				except:	
					IDF_dict[find] = IDF_dict.get(find,1)
			word_set.add(find)

	word_df = pd.DataFrame(list(zip(ham_TF_dict.keys(),ham_TF_dict.values(),spam_TF_dict.values(),IDF_dict.values())))
	word_df.columns = ['keyword','ham_TF','spam_TF','IDF']
	word_df['ham_TF'] = word_df['ham_TF'].astype('float')/train_df[train_df['label']=='ham'].shape[0]
	word_df['spam_TF'] = word_df['spam_TF'].astype('float')/train_df[train_df['label']=='spam'].shape[0]
	word_df['IDF'] = np.log10(train_df.shape[0]/word_df['IDF'].astype('float'))
	word_df['ham_TFIDF'] = word_df['ham_TF']*word_df['IDF']
	word_df['spam_TFIDF'] = word_df['spam_TF']*word_df['IDF']
	word_df['diff']=word_df['spam_TFIDF']-word_df['ham_TFIDF']

	selected_spam_key = word_df.sort_values('diff',ascending=False)

	print('>>>Generating Tabu List...\n  Tabu List Size: {}\n  File Name: {}\n  The words shorter than {} are ignored by model\n'.format(tabu_size, path, ignore))
	file = open(path,'w')
	for word in selected_spam_key.head(tabu_size).keyword:
		file.write(word+'\n')
	file.close()

4. Read Tabu List and Convert SMS
-----------------------------------
Since the message is of variant length, it is not easy for the implementation of learning algorithm. So we define a Function above generating tabu list and storing them in the local file. And we can use this file to convert a SMS expressed in string to a vector of fixed length expressed in binary value.

The idea is given like this: If we have a tabu list then we could find those word in the list and represent them by a index. Thus a string can be converted to an array of int. Further, we could define an array filled with zeros with the same length of tabu list. if this str contains the word in the tabu list, we could assign 1 to the corresponding element of the array representing 'message contains word w'. (tips: the query of `python.dict` is of constant time, much faster than `python.list`)

By taking this step, we could convert our raw data of variant length into the numeric data of fixed length.

These two function is given below:

In [ ]:
def read_tabu_list():
	file = open('tabu.txt','r')
	keyword_dict = dict()
	i = 0
	for line in file:
		keyword_dict.update({line.strip():i})
		i+=1
	return keyword_dict

def convert_Content(content, tabu):
	m = len(tabu)
	res = np.int_(np.zeros(m))
	finds = re.findall('[A-Za-z]+', content)
	for find in finds:
		find=find.lower()
		try:
			i = tabu[find]
			res[i]=1
		except:
			continue
	return res

5. Learning, Testing and Predicting
-----------------------------------
After we generate our tabu list and those supporting functions, we are now well prepared for the learning part in this problem. And here we could use the library `from sklearn.naive_bayes import BernoulliNB`. It will help us train this model.

Before this part, let review our data: our feature input X is a n*m matrix, where X[i,j] = 1 means the sample #i contains the word j in the tabu list, and supervised label Y is a n*1 vector where Y[i] = 1 representing for a spam and 0 for a ham.

Let prepare the materials for the learning algorithm.

In [ ]:
def learn():
	global tabu, m
	train,_ = readData()
	n = train.shape[0]
	X = np.zeros((n,m)); Y=np.int_(train.label=='spam')
	for i in range(n):
		X[i,:] = convert_Content(train.iloc[i].content, tabu)

	NaiveBayes = BernoulliNB()
	NaiveBayes.fit(X, Y)

	Y_hat = NaiveBayes.predict(X)
	print('>>>Learning...\n  Learning Sample Size: {}\n  Accuarcy (Training sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
	return NaiveBayes

The Function above returns a well trained Naive Bayes Model object, and we could use it to make prediction.

In [ ]:
def test(NaiveBayes):
	global tabu, m
	_,test = readData()
	n = test.shape[0]
	X = np.zeros((n,m)); Y=np.int_(test.label=='spam')
	for i in range(n):
		X[i,:] = convert_Content(test.iloc[i].content, tabu)
	Y_hat = NaiveBayes.predict(X)
	print ('>>>Cross Validation...\n  Testing Sample Size: {}\n  Accuarcy (Testing sample): {:.2f}％\n'.format(n,sum(np.int_(Y_hat==Y))*100./n))
	return

def predictSMS(SMS):
	global NaiveBayes, tabu, m
	X = convert_Content(SMS, tabu)
	Y_hat = NaiveBayes.predict(X.reshape(1,-1))
	if int(Y_hat) == 1:
		print ('SPAM: {}'.format(SMS))
	else:
		print ('HAM: {}'.format(SMS))

6. Overall Assembly
-------------------
After we define the every modules we need in this problem, we could integrate them into a whole part.

In [ ]:
print('UCI SMS SPAM CLASSIFICATION PROBLEM SET\n  -- implemented by Bernoulli Naive Bayes Model\n')
tabu_file = 'tabu.txt'          # user defined tabu file
tabu_size = 300                 # how many features are used to classify spam
word_len_ignored = 3            # ignore those words shorter than this variable
# build a tabu list based on the training data
generate_tabu_list(tabu_file,tabu_size, word_len_ignored)

tabu = read_tabu_list()
m = len(tabu)
# train the Naive Bayes Model using training data
NaiveBayes=learn()
# Test Model using testing data
test(NaiveBayes)
print('>>>Testing')
# I select two messages from the test data here.
predictSMS('Ya very nice. . .be ready on thursday')
predictSMS('Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call')

Okay, it works! A accuracy of 98.28% in training set and 97.77% is acceptable for me.

Pleas feel free to ask me, if you have any question. And if you like this guide, please upvote, Thanks a lot.

--Clyde Wang